# RAG re-ranking
#### Sfruttamento della comprensione di un LLM per la selezione dei documenti selezionati dal Vector Store tramite Retriever

In [14]:
# Importazioni necessarie
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from typing import List
import re
import os
from pydantic import BaseModel, Field

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
# Funzione per caricare documenti di testo
def load_documents(file_path: str) -> List[str]:
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().split('\n\n')

In [4]:
# Funzione per creare il Vector Store
def create_vector_store(documents: List[str]):
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    texts = text_splitter.create_documents(documents)
    
    embeddings = HuggingFaceEmbeddings()
    vector_store = Chroma.from_documents(texts, embeddings)
    return vector_store

In [5]:
# Funzione per il retrieval iniziale
def initial_retrieval(query: str, vector_store, k=10):
    return vector_store.similarity_search(query, k=k)

In [6]:
class Relevance(BaseModel):
    """Relevance of a document to a query"""

    relevance: int = Field(default="no", description="a value between 1 and 10 that indicates the relevance of the document to the query")

In [7]:
# Funzione per il reranking
def rerank_documents(query: str, documents: List[str], llm, verbose=False):
    rerank_prompt = ChatPromptTemplate([
        ("system", "Rate the relevance of a document to a query with a score from 1 to 10"),
        ("user", "Document:\n{document}\n\nQuery:\n{query}")
    ])
    
    rerank_chain = rerank_prompt | llm.with_structured_output(Relevance)
    
    scores = []
    for doc in documents:
        doc_content = doc.page_content
        response = rerank_chain.invoke({"query":query, "document":doc_content})
        score = response.relevance
        scores.append((doc, score))
        if verbose:
            print(doc_content[:50], score)
    
    return sorted(scores, key=lambda x: x[1], reverse=True)  # ritorno la lista di documenti ordinata per rilevanza 
                                                             # rispetto alla query dell'utente

In [8]:
# Funzione principale RAG
def rag_with_reranking(query: str, vector_store, llm, verbose=False):
    # Retrieval iniziale
    initial_results = initial_retrieval(query, vector_store)

    # Reranking
    reranked_results = rerank_documents(query, initial_results, llm, verbose)
    if verbose:
        print("----------- RANKED RESULTS -----------")
        print(reranked_results)
        print("--------------------------------------")
    # Generazione della risposta sfruttando i 3 documenti più rilevanti tra i 10 selezionati
    context = "\n".join([doc.page_content for doc, _ in reranked_results[:3]])
    
    answer_prompt = PromptTemplate(
        input_variables=["query", "context"],
        template="Basandoti sul seguente contesto, rispondi alla domanda.\n\nContesto: {context}\n\nDomanda: {query}\n\nRisposta:"
    )
    
    answer_chain = answer_prompt | llm
    answer = answer_chain.invoke({"query": query, "context": context})
    
    return answer

In [9]:
# Caricamento dei documenti
documents = load_documents("data/testo_esempio.txt")

In [10]:
print(f"{len(documents)} split del documento su `\\n\\n` caricati")

25 split del documento su `\n\n` caricati


In [11]:
# Creazione del vector store
vector_store = create_vector_store(documents)

In [15]:
# Esecuzione della RAG con reranking
query = "Quale evento inaspettato costringe Sofia a tornare nella sua città natale durante il suo percorso universitario a Milano?"  # L'ictus di sua nonna
answer = rag_with_reranking(query, vector_store, llm, verbose=True)

Nei giorni successivi Sofia si ritrovò a dover aff 5
Ma come spesso accade, proprio quando tutto sembra 10
La vita universitaria la stava cambiando profondam 2
I mesi volavano e Sofia si sentiva sempre più part 2
Da quel momento Marco entrò prepotentemente nella  2
Era una tiepida mattina di primavera quando Sofia  3
Il ritorno a Milano fu più difficile del previsto. 2
L'autunno portò con sé nuovi corsi all'università  2
Quando finalmente arrivò alla stazione centrale di 2
Fu in quei giorni difficili che Sofia si rese cont 7
----------- RANKED RESULTS -----------
[(Document(id='804a20c9-adf2-414a-a8f0-31cbc6939e56', metadata={}, page_content="Ma come spesso accade, proprio quando tutto sembra andare per il meglio la vita ti mette davanti a degli ostacoli. Una sera, mentre tornava dal lavoro, Sofia ricevette una telefonata che le gelò il sangue nelle vene. Sua nonna aveva avuto un ictus ed era stata ricoverata d'urgenza. Senza pensarci due volte, Sofia fece una valigia e prese il primo 

In [16]:
print(f"Domanda: {query}")
print(f"Risposta: {answer.content}")

Domanda: Quale evento inaspettato costringe Sofia a tornare nella sua città natale durante il suo percorso universitario a Milano?
Risposta: Sofia è costretta a tornare nella sua città natale a causa di un evento inaspettato: sua nonna ha avuto un ictus ed è stata ricoverata d'urgenza.


In [17]:
query = "In quale quartiere di Milano si trova il bistrot dove Sofia trova lavoro come cameriera part-time?"  # Brera
answer = rag_with_reranking(query, vector_store, llm)

print(f"Domanda: {query}")
print(f"Risposta: {answer.content}")

Domanda: In quale quartiere di Milano si trova il bistrot dove Sofia trova lavoro come cameriera part-time?
Risposta: Il bistrot dove Sofia trova lavoro come cameriera part-time si trova nel quartiere di Brera.


In [18]:
query = "Come si chiamano le due coinquiline con cui Sofia condivide l'appartamento a Milano?"  # Giulia e Martina
answer = rag_with_reranking(query, vector_store, llm)

print(f"Domanda: {query}")
print(f"Risposta: {answer.content}")

Domanda: Come si chiamano le due coinquiline con cui Sofia condivide l'appartamento a Milano?
Risposta: Le due coinquiline con cui Sofia condivide l'appartamento a Milano si chiamano Giulia e Martina.
